In [31]:
import requests
import json
import sqlite3
import time
import folium
import pandas as pd

#create our own database
conn = sqlite3.connect('traffic_data.db')  
c = conn.cursor()
#creates a table anmed "incidents" if it doesn't already exist
#the table has columns for incdient details such as ID,type, severity, description, latitude, longitude 
c.execute("CREATE TABLE IF NOT EXISTS incidents (id INTEGER PRIMARY KEY, type TEXT, severity INTEGER, description TEXT, lat REAL, lng REAL)")
conn.commit()
conn.close()

def insert_data(data):
    #defined to insert incident data inot the "incidents" table
    conn = sqlite3.connect('traffic_data.db')
    c = conn.cursor()
    #takes a list of data tuples as input and insert multiple rows into the table at once
    c.executemany("INSERT INTO incidents (id, type, severity, description, lat, lng) VALUES (?, ?, ?, ?, ?, ?)", data)
    conn.commit()
    conn.close()

def get_traffic_data(bbox):
    #retrieves traffic incident data from MapQuest Traffic API
    key = "Sks7L0lksbFj0xPNyBdglVFjmsAJGJCU"
    response = requests.get(f"https://www.mapquestapi.com/traffic/v2/incidents?key={key}&boundingBox={bbox}&filters=congestion,incidents,construction,event")
    data = response.json()
    return data

def store_traffic_data():
    #store traffic incident data in the database. 
    bbox_step = 0.1  # Bbox step size for iterating over the U.S. 
    bbox_range = {
        # Starting latitude for bbox
        "lat_start": 24.396308,  #southernmost
        # Ending latitude for bbox
        "lat_end": 49.384358,    #northernmost
        # Starting longitude for bbox
        "lng_start": -125.000000,  #westernmost
        # Ending longitude for bbox
        "lng_end": -66.934570    #easternmost
    }
    create_table()
    bbox = {
        "lat_start": bbox_range["lat_start"],
        "lat_end": bbox_range["lat_start"] + bbox_step,
        "lng_start": bbox_range["lng_start"],
        "lng_end": bbox_range["lng_start"] + bbox_step
    }
    page = 1
    # a loop that continues until the latitude of the current 
    #bounding box exceeds the northernmost latitude of the bbox_range.
    while bbox["lat_start"] <= bbox_range["lat_end"]:
        #to fetch traffic incident data for the current bounding box
        response_data = get_traffic_data(f"{bbox['lat_start']},{bbox['lng_start']},{bbox['lat_end']},{bbox['lng_end']}")
        incidents = response_data.get("incidents")
        if not incidents:
            break
            #If there are no incidents, it breaks out of the loop, assuming that there is no more data to retrieve.
        data = []
        #extracting relevant information from each incident and storing it as a tuple in the data list
        for incident in incidents:
            incident_id = incident.get("id")
            incident_type = incident.get("type")
            incident_severity = incident.get("severity")
            incident_description = incident.get("shortDesc")
            incident_lat = incident.get("lat")
            incident_lng = incident.get("lng")
            data.append((incident_id, incident_type, incident_severity, incident_description, incident_lat, incident_lng))
        #inserting the incident data into the database.
        insert_data(data)
        print(f"Page {page} processed.")
        page += 1
        #the longitude values of the bounding box are updated by adding the bbox_step 
        #value to both the starting and ending longitudes
        bbox["lng_start"] += bbox_step
        bbox["lng_end"] += bbox_step
        #If the updated longitude exceeds the easternmost longitude of the bbox_range, 
        #the latitude values are updated, and the longitude values are reset to the starting values
        if bbox["lng_start"] > bbox_range["lng_end"]:
            bbox["lat_start"] += bbox_step
            bbox["lat_end"] += bbox_step
            bbox["lng_start"] = bbox_range["lng_start"]
            bbox["lng_end"] = bbox_range["lng_start"] + bbox_step
        time.sleep(1)  # Sleep for 1 second to avoid hitting API rate limits

def get_incidents_in_area(bbox):
    conn = sqlite3.connect('traffic_data.db')
    c = conn.cursor()
    c.execute("SELECT * FROM incidents WHERE lat BETWEEN ? AND ? AND lng BETWEEN ? AND ?", bbox)
    incidents = c.fetchall()
    conn.close()
    return incidents

def display_map_with_incidents(incidents):
    # Create an empty map centered around the first incident
    
    if incidents:
        center_lat, center_lng = incidents[0][4], incidents[0][5]
        #[0]: incident; [4]: latitude; [5]: longitude
    else:
        center_lat, center_lng = 0, 0
    map_traffic = folium.Map(location=[center_lat, center_lng], zoom_start=10)

    # Add markers for each incident
    for incident in incidents:
        #the location parameter set to [incident_lat, incident_lng], 
        #representing the coordinates of the incident
        incident_lat, incident_lng = incident[4], incident[5]
        marker = folium.Marker(location=[incident_lat, incident_lng])
        marker.add_to(map_traffic)

    return map_traffic



In [32]:
bbox = (-118.0046, -117.9851, 33.8352, 33.8828)  

incidents = get_incidents_in_area(bbox)
map_with_incidents = display_map_with_incidents(incidents)
map_with_incidents